In [4]:
import numpy as np
import math
import scipy.stats as stat
import matplotlib.pyplot as plt
import pandas as pd
import os
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot, plot_mpl
init_notebook_mode(connected=True)
path = r'C:\\Users\\inventor\\Desktop\\FR study\\FR-30th\\Derivatives_study\\FR_30_NIH\\binomial_modeling'
os.chdir(path)
 
# 위험 중립 세계에서 u와 d의 변동성을 일치 시킴(12.7)
# 배당이 없는(무배당) 유러피안 옵션의 1기간 이항모형
def Binomial_European_One_period(option_type, S, K, u, rf):
    d = 1/u
    t=1                                  # t = maturity
    n=1                                  # n 은 기간 : 1 period
    dt = t/n
    a = np.exp(rf * dt)                  # rf는 무위험 이자율
    df = 1/np.exp(rf * dt)               # df는 discount factor 할인계수
    prob = (a-d)/(u-d)
    
    if option_type == 'call':
        cu = max(S*u - K, 0)                
        cd = max(S*d - K, 0)
        c = (prob*cu + (1-prob)*cd) * df
        
        return c
    else:
        pu = max(K - S*u, 0)                
        pd = max(K - S*d, 0)
        p = (prob*pu + (1-prob)*pd) * df

        return p

print(Binomial_European_One_period(option_type='call', S=100, K=100, u=1.1, rf=0.05)) 
print(Binomial_European_One_period(option_type='put', S=100, K=100, u=1.1, rf=0.05))  

S = np.linspace(0, 200, 1000)
Call_Value = np.zeros(len(S))
Put_Value = np.zeros(len(S))

for i in range(len(S)):
    Call_Value[i] = Binomial_European_One_period(option_type='call', S=S[i], K=100, u=1.1, rf=0.05)
    Put_Value[i] = Binomial_European_One_period(option_type='put', S=S[i], K=100, u=1.1, rf=0.05)

trace1 = go.Scatter(x=S, y=Call_Value)
data1=[trace1]
trace2 = go.Scatter(x=S, y=Put_Value)
data2=[trace2]
layout = go.Layout(width=800, height=400, xaxis=dict(title='Spot Price'), yaxis=dict(title='Option Value')) 
fig1 = dict(data=data1, layout=layout)
fig2 = dict(data=data2, layout=layout)
iplot(fig1, image='png')
iplot(fig2, image='png')

# 오프라인에 그림 저장하기 https://chart-studio.plotly.com/~notebook_demo/122.embed


7.084313119013628
2.2072555690850195


In [2]:
# 위험 중립 세계에서 u와 d의 변동성을 일치 시킴(12.7)
# 배당이 없는(무배당) 유러피안 옵션의 2기간 이항모형(12.3)
def Binomial_European_Two_period(option_type, S, K, u, rf):
    d = 1/u
    t=1                                  # t = maturity
    n=2                                  # n 은 기간 : 2 period
    dt = t/n
    a = np.exp(rf * dt)                  # rf는 무위험 이자율
    df = 1/np.exp(rf * dt)               # df는 discount factor 할인계수
    prob = (a-d)/(u-d)
    
    if option_type == 'call':
        cuu = max(S*u*u - K, 0)
        cud = max(S*u*d - K, 0)
        cdd = max(S*d*d - K, 0)
        
        cu = ((prob * cuu) + ((1-prob) * cud)) * df
        cd = ((prob * cud) + ((1-prob) * cdd)) * df
        
        c = ((prob* cu) + ((1-prob) * cd)) * df
        
        return c
    else :
        puu = max(K - S*u*u, 0)
        pud = max(K - S*u*d, 0)
        pdd = max(K - S*d*d, 0)
        
        pu = ((prob * puu) + ((1-prob) * pud)) * df
        pd = ((prob * pud) + ((1-prob) * pdd)) * df
        
        p = ((prob* pu) + ((1-prob) * pd)) * df

        return p

print(Binomial_European_Two_period(option_type='call', S=100, K=100, u=1.1, rf=0.05))
print(Binomial_European_Two_period(option_type='put', S=100, K=100, u=1.1, rf=0.05))

S = np.linspace(0, 200, 1000)
Call_Value = np.zeros(len(S))
Put_Value = np.zeros(len(S))

for i in range(len(S)):
    Call_Value[i] = Binomial_European_Two_period(option_type='call', S=S[i], K=100, u=1.1, rf=0.05)
    Put_Value[i] = Binomial_European_Two_period(option_type='put', S=S[i], K=100, u=1.1, rf=0.05)

trace1 = go.Scatter(x=S, y=Call_Value)
data1=[trace1]
trace2 = go.Scatter(x=S, y=Put_Value)
data2=[trace2]
layout = go.Layout(width=800, height=400, xaxis=dict(title='Spot Price'), yaxis=dict(title='Option Value')) 
fig1 = dict(data=data1, layout=layout)
fig2 = dict(data=data2, layout=layout)
iplot(fig1, image='png')
iplot(fig2, image='png')

7.40362763732484
2.526570087396231


In [3]:
# 위험 중립 세계에서 u와 d의 변동성을 일치 시킴(12.7)
# 배당수익률 q인 주가지수 유러피안 옵션의 n기간 이항모형 (12.11)
def Binomial_European_n_period(option_type, S, K, rf, T, n, sigma, q, vu=0):
    dt = T / n                              # T : maturity, n : period
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u                               
    a = np.exp((rf - q) * dt)               # rf는 무위험 이자율, q는 연속배당수익률
    df = 1 / np.exp(rf * T)                 # df는 discount factor 할인계수
    prob = (a - d) / (u - d)
    
    if option_type == 'call':
        for i in range(n+1):
            st = S * (u ** i) * (d ** (n - i))
            vu = vu + stat.binom(n, prob).pmf(i) * max(st - K, 0) * df

        return vu
    else:
        for i in range(n+1):
            st = S * (u ** i) * (d ** (n - i))
            vu = vu + stat.binom(n, prob).pmf(i) * max(K - st, 0) * df

        return vu

print(Binomial_European_n_period(option_type='call', S=100, K=100, rf=0.05, T=1, n=1000, sigma=0.3, q=0))
print(Binomial_European_n_period(option_type='put', S=100, K=100, rf=0.05, T=1, n=1000, sigma=0.3, q=0))

# S = np.linspace(0, 200, 1000)
# Call_Value = np.zeros(len(S))
# Put_Value = np.zeros(len(S))

# for i in range(len(S)):
#     Call_Value[i] = Binomial_European_n_period(option_type='call', S=S[i], K=100, rf=0.05, T=1, n=1000, sigma=0.3, q=0)
#     Put_Value[i] = Binomial_European_n_period(option_type='put', S=S[i], K=100, rf=0.05, T=1, n=1000, sigma=0.3, q=0)

# trace1 = go.Scatter(x=S, y=Call_Value)
# data1=[trace1]
# trace2 = go.Scatter(x=S, y=Put_Value)
# data2=[trace2]
# layout = go.Layout(width=800, height=400, xaxis=dict(title='Spot Price'), yaxis=dict(title='Option Value')) 
# fig1 = dict(data=data1, layout=layout)
# fig2 = dict(data=data2, layout=layout)
# iplot(fig1, image='png')
# iplot(fig2, image='png')

14.228309015844891
9.351251465911211


In [1]:
import pandas as pd
import os
path = r'C:\\Users\\inventor\\Desktop\\FR study\\FR-30th\\Derivatives_study\\FR_30_NIH\\binomial_modeling'
os.chdir(path)
name = 'make a simple tree'
 
# n칸 짜리 tree만들기
n=10
k=0
l=0
n_col = [x for x in range(0, 50)]
n_row = [x for x in range(0, 50)]
df = pd.DataFrame(columns=n_col, index=n_row)
 
for y in range(n, 0, -1):
    for x in range(y, 0, -1):
        k+=1
        l+=10
        df.loc[4*x-3 + 2*(n-y), y] = k
        df.loc[4*x-4 + 2*(n-y), y] = l
 
df.dropna(how='all', inplace=True, axis=0)
df.dropna(how='all', inplace=True, axis=1)
 
df.to_excel('make a simple tree.xlsx', sheet_name=name)
pd.options.display.max_columns = None
pd.options.display.max_rows = None
# df

,1,2,3,4,5,6,7,8,9,10
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,190,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,270,NaN,90
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27,NaN,9
6,NaN,NaN,NaN,NaN,NaN,NaN,340,NaN,180,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,34,NaN,18,NaN
8,NaN,NaN,NaN,NaN,NaN,400,NaN,260,NaN,80
9,NaN,NaN,NaN,NaN,NaN,40,NaN,26,NaN,8


In [31]:
# 아메리칸 콜옵션을 이항 모델로 계산하기. 간단한 예제 만들어서 기초자산 가격이랑 옵션 가격 tree 형태로 표시
S=100
K=100
rf=0.05
T=1
n=5
u=1.1
d=1/u
dt = T/n
r = np.exp(rf * dt)
dfr = 1/np.exp(rf * dt)
prob = (r-d)/(u-d)
 
n_col = [x for x in range(0, 200)]
n_row = [x for x in range(0, 200)]
df = pd.DataFrame(columns=n_col, index=n_row)
 
n=5
 
for y in range(n, 0-1, -1):
    for x in range(y, 0-1, -1):
 
        if y == n:
            z = max((u ** (y-x)) * (d ** x) * S - K, 0)
        
        elif 0 < y < n:
            z = max((u ** (y-x)) * (d ** x) * S - K,
                    ((prob * df.loc[4*x + 2*(n-y) - 1, y+2]) + ((1-prob)*df.loc[4*x+2*(n-y) + 3, y+2]))*dfr)
        else:
            z = ((prob * df.loc[4*x +2*(n-y) - 1, y+2]) + ((1-prob )* df.loc[4*x+2*(n-y) + 3, y+2]))*dfr
        
        st = (u ** (y-x)) * (d ** x) * S

        df.loc[4*x + 2*(n-y) + 1, y+1] = z
        df.loc[4*x-1 + 2*(n-y) + 1, y+1] = st
 
df.dropna(how='all', inplace=True, axis=0)
df.dropna(how='all', inplace=True, axis=1)
df = df.reset_index(drop='index')
df.columns = range(df.shape[1])
name = 'binomial model'
df.to_excel('american option binomial model.xlsx', sheet_name=name)
pd.options.display.max_columns = None
pd.options.display.max_rows = None
# df

In [32]:
# 유로피언 옵션을 이항 모델로 계산하기. 간단한 예제 만들어서 기초자산 가격이랑 옵션 가격 tree 형태로 표시
# 예제 12.1, 그림 12.11
S=810
K=800
rf=0.05
T=0.5
n=2
sigma=0.2
q=0.02
option_type='call'

dt = T / n                              # T : maturity, n : period
u = np.exp(sigma * np.sqrt(dt))
d = 1 / u                               
a = np.exp((rf - q) * dt)               # rf는 무위험 이자율, q는 연속배당수익률
dfr = 1 / np.exp(rf * dt)                 # df는 discount factor 할인계수
prob = (a - d) / (u - d)
 
n_col = [x for x in range(0, 200)]
n_row = [x for x in range(0, 200)]
df = pd.DataFrame(columns=n_col, index=n_row)
 
for y in range(n, 0-1, -1):
    for x in range(y, 0-1, -1):

        st = (u ** (y-x)) * (d ** x) * S
        
        if y == n:
            if option_type== 'call':
                z = max(st - K, 0)
            else:
                z = max(K - st, 0)
        
        elif 0 < y < n:
            z = ((prob * df.loc[4*x + 2*(n-y) - 1, y+2]) + ((1-prob)*df.loc[4*x+2*(n-y) + 3, y+2]))*dfr

        else : 
            z = ((prob * df.loc[4*x +2*(n-y) - 1, y+2]) + ((1-prob)* df.loc[4*x+2*(n-y) + 3, y+2]))*dfr


        df.loc[4*x + 2*(n-y) + 1, y+1] = z
        df.loc[4*x-1 + 2*(n-y) + 1, y+1] = st
 
df.dropna(how='all', inplace=True, axis=0)
df.dropna(how='all', inplace=True, axis=1)
df = df.reset_index(drop='index')
df.columns = range(df.shape[1])
name = 'fig 12.11'
df.to_excel('europian option binomial model.xlsx', sheet_name=name)
pd.options.display.max_columns = None
pd.options.display.max_rows = None
df

,0,1,2
0,NaN,NaN,989.336
1,NaN,NaN,189.336
2,NaN,895.188,NaN
3,NaN,100.661,NaN
4,810,NaN,810
5,53.3947,NaN,10
6,NaN,732.918,NaN
7,NaN,5.06232,NaN
8,NaN,NaN,663.172
9,NaN,NaN,0


In [36]:
# 아메리칸 옵션을 이항 모델로 계산하기. 간단한 예제 만들어서 기초자산 가격이랑 옵션 가격 tree 형태로 표시
# 예제 12.2, 그림 12.12
S=0.06100
K=0.06000
rf=0.07
r=0.05
T=3/12
n=3
sigma=0.12
option_type='call'

dt = T / n                              # T : maturity, n : period
u = np.exp(sigma * np.sqrt(dt))
d = 1 / u                               
a = np.exp((r - rf) * dt)               # rf는 foreign risk free rate, r는 domestic risk free rate
dfr = 1 / np.exp(r * dt)                # df는 discount factor 할인계수
prob = (a - d) / (u - d)
n_col = [x for x in range(0, 200)]
n_row = [x for x in range(0, 200)]
df = pd.DataFrame(columns=n_col, index=n_row)
 
 
for y in range(n, 0-1, -1):
    for x in range(y, 0-1, -1):
 
        st = (u ** (y-x)) * (d ** x) * S
        
        if y == n:
            if option_type== 'call':
                z = max(st - K, 0)
            else:
                z = max(K - st, 0)
        
        elif 0 < y < n:
            if option_type== 'call':
                z = max(st - K,
                    ((prob * df.loc[4*x + 2*(n-y) - 1, y+2]) + ((1-prob)*df.loc[4*x+2*(n-y) + 3, y+2]))*dfr)
            else:
                z = max(K - st,
                    ((prob * df.loc[4*x + 2*(n-y) - 1, y+2]) + ((1-prob)*df.loc[4*x+2*(n-y) + 3, y+2]))*dfr)  

        else:
            z = (prob * df.loc[4*x +2*(n-y) - 1, y+2]) + ((1-prob )* df.loc[4*x+2*(n-y) + 3, y+2])*dfr
        

        df.loc[4*x + 2*(n-y) + 1, y+1] = z
        df.loc[4*x-1 + 2*(n-y) + 1, y+1] = st
 
df.dropna(how='all', inplace=True, axis=0)
df.dropna(how='all', inplace=True, axis=1)
df = df.reset_index(drop='index')
df.columns = range(df.shape[1])
name = 'fig 12.12'
df.to_excel('american option of currency.xlsx', sheet_name=name)
pd.options.display.max_columns = None
pd.options.display.max_rows = None
df

,0,1,2,3
0,NaN,NaN,NaN,0.0676804
1,NaN,NaN,NaN,0.00768042
2,NaN,NaN,0.065376,NaN
3,NaN,NaN,0.00537604,NaN
4,NaN,0.0631501,NaN,0.0631501
5,NaN,0.00327949,NaN,0.00315013
6,0.061,NaN,0.061,NaN
7,0.00189443,NaN,0.00146596,NaN
8,NaN,0.0589231,NaN,0.0589231
9,NaN,0.00068221,NaN,0


In [37]:
# 아메리칸 옵션을 이항 모델로 계산하기. 간단한 예제 만들어서 기초자산 가격이랑 옵션 가격 tree 형태로 표시
# 예제 12.3, 그림 12.13
S=31                                    # S=F 로 선물가격
K=30
rf=0.05
q=rf                                    # 선물옵션의 경우
T=9/12
n=3
sigma=0.30
option_type='put'

dt = T / n                              # T : maturity, n : period
u = np.exp(sigma * np.sqrt(dt))
d = 1 / u                               
a = np.exp((rf - q) * dt)                # rf=1가 되서 a=1 이 됨.
dfr = 1 / np.exp(rf * dt)                # df는 discount factor 할인계수
prob = (a - d) / (u - d)

n_col = [x for x in range(0, 200)]
n_row = [x for x in range(0, 200)]
df = pd.DataFrame(columns=n_col, index=n_row)
 
 
for y in range(n, 0-1, -1):
    for x in range(y, 0-1, -1):
 
        st = (u ** (y-x)) * (d ** x) * S
        
        if y == n:
            if option_type== 'call':
                z = max(st - K, 0)
            else:
                z = max(K - st, 0)
        
        elif 0 < y < n:
            if option_type== 'call':
                z = max(st - K,
                    ((prob * df.loc[4*x + 2*(n-y) - 1, y+2]) + ((1-prob)*df.loc[4*x+2*(n-y) + 3, y+2]))*dfr)
            else:
                z = max(K - st,
                    ((prob * df.loc[4*x + 2*(n-y) - 1, y+2]) + ((1-prob)*df.loc[4*x+2*(n-y) + 3, y+2]))*dfr)  

        else:
            z = (prob * df.loc[4*x +2*(n-y) - 1, y+2]) + ((1-prob )* df.loc[4*x+2*(n-y) + 3, y+2])*dfr
        

        df.loc[4*x + 2*(n-y) + 1, y+1] = z
        df.loc[4*x-1 + 2*(n-y) + 1, y+1] = st
 
df.dropna(how='all', inplace=True, axis=0)
df.dropna(how='all', inplace=True, axis=1)
df = df.reset_index(drop='index')
df.columns = range(df.shape[1])
name = 'fig 12.13'
df.to_excel('american option of futures.xlsx', sheet_name=name)
pd.options.display.max_columns = None
pd.options.display.max_rows = None
df

,0,1,2,3
0,NaN,NaN,NaN,48.6177
1,NaN,NaN,NaN,0
2,NaN,NaN,41.8456,NaN
3,NaN,NaN,0,NaN
4,NaN,36.0169,NaN,36.0169
5,NaN,0.934694,NaN,0
6,31,NaN,31,NaN
7,2.84101,NaN,1.76107,NaN
8,NaN,26.6819,NaN,26.6819
9,NaN,4.53816,NaN,3.31805
